# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
sum = 0
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    sum += sqrt_value

print (sum)
generator2 = square_root_generator(13)

data = list(generator2)
print(data[-1])

8.382332347441762
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [8]:
#Load the first generator
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [ ]:
!pip install dlt[duckdb]

In [12]:
import dlt
# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
pipeline = dlt.pipeline(pipeline_name="people_data",
						destination='duckdb',
						dataset_name='people')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1,
                    table_name="people1",
                    write_disposition="replace")

# show the outcome
print(info)

Pipeline people_data load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/people_data.duckdb location to store data
Load package 1708278777.0774739 is LOADED and contains no failed jobs


In [21]:
# calculate the sum of ages of all people

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n the sum of ages of all people:")

the_people = conn.sql("SELECT SUM(age) FROM people1").df()
display(the_people)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people1             │
└─────────────────────┘




 the sum of ages of all people:


,sum(age)
0,353.0


In [18]:
# Append the second generator to the same table as the first.
info = pipeline.run(people_2,
										table_name="people1",
										write_disposition="append")

print(info)


Pipeline people_data load step completed in 0.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/people_data.duckdb location to store data
Load package 1708283754.4614213 is LOADED and contains no failed jobs


In [22]:
#After correctly appending the data, calculate the sum of all ages of people.


print("\n\n\n the sum of all ages of people:")

the_people = conn.sql("SELECT SUM(age) FROM people1").df()
display(the_people)





 the sum of all ages of people:


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [23]:
# define the connection to load to.
pipeline2 = dlt.pipeline(pipeline_name="people_data_new", destination='duckdb', dataset_name='people_new')


# run with merge write disposition.
# This is so scaffolding is created for the next example,
# where we look at merging data

info = pipeline2.run(people_1,
										table_name="people",
										write_disposition="replace",
                    primary_key="ID")

print(info)

Pipeline people_data_new load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_new
The duckdb destination used duckdb:////content/people_data_new.duckdb location to store data
Load package 1708284648.4761329 is LOADED and contains no failed jobs


In [24]:
# show outcome

conn = duckdb.connect(f"{pipeline2.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline2.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n People table below:")

people = conn.sql("SELECT * FROM people").df()
display(people)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘




 People table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708284648.4761329,md4FkWL8OrDG+Q
1,2,Person_2,27,City_A,1708284648.4761329,Oo+SOf5gSLx7+g
2,3,Person_3,28,City_A,1708284648.4761329,nCJgvWIx8mD47Q
3,4,Person_4,29,City_A,1708284648.4761329,dKR+0waaXl4q4A
4,5,Person_5,30,City_A,1708284648.4761329,7PgHrLrkIty+PQ


In [25]:
#then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

#Question: Calculate the sum of ages of all the people loaded as described above.

info = pipeline2.run(people_2,
										table_name="people",
										write_disposition="merge",
                    primary_key="ID")

print(info)

Pipeline people_data_new load step completed in 0.70 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_new
The duckdb destination used duckdb:////content/people_data_new.duckdb location to store data
Load package 1708284932.4481792 is LOADED and contains no failed jobs


In [28]:
#After loading, you should have a total of 8 records, and ID 3 should have age 33.

print("\n\n\n People table below:")

people = conn.sql("SELECT COUNT(*) FROM people").df()
display(people)




 People table below:


,count_star()
0,8


In [29]:
#Question: Calculate the sum of ages of all the people loaded as described above.
print("\n\n\n sum of ages of all the people loaded:")

sum_age = conn.sql("SELECT SUM(age) FROM people").df()
display(sum_age)




 sum of ages of all the people loaded:


,sum(age)
0,266.0


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX